In [72]:
%matplotlib inline

In [73]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [74]:
SOS_token = 0
EOS_token = 1

In [75]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.MAX_LENGTH = 2
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        words = sentence.split(' ')
        self.MAX_LENGTH = 2+max(self.MAX_LENGTH-2,len(words))
        for word in words:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    
    def addWords(self, words):
        for word in words:
            addWord(word)

In [76]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [77]:
def readLangs(articles_file, titles_file):
    print("Reading lines...")

    # Read the file and split into lines
    articles_lines = open('data/'+articles_file, encoding='utf-8').read().strip().split('\n')
    titles_lines = open('data/'+titles_file, encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    articles_lines = [normalizeString(s) for s in articles_lines]
    titles_lines = [normalizeString(s) for s in titles_lines]

    pairs= []
    
    for i in range(len(articles_lines)):
        pair = [articles_lines[i],titles_lines[i]+" ."]
        pairs.append(pair)
    
    sent = Lang("sentence")
    summ = Lang("summary")

    return sent, summ, pairs

In [78]:
def prepareData(articles_file, titles_file):
    sent, summ, pairs = readLangs(articles_file,titles_file)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        sent.addSentence(pair[0])
        summ.addSentence(pair[1])
    print("Counted words:")
    print(sent.name, sent.n_words)
    print(summ.name, summ.n_words)
    return sent, summ, pairs

In [95]:
sent, summ, pairs = prepareData("mini.art.txt","mini.tit.txt")
print(sent.MAX_LENGTH, summ.MAX_LENGTH)

Reading lines...
Read 20 sentence pairs
Counting words...
Counted words:
sentence 285
summary 102
40 14


In [80]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [81]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [82]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=50):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [83]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(sent, pair[0])
    target_tensor = tensorFromSentence(summ, pair[1])
    return (input_tensor, target_tensor)

In [84]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
        
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [85]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [86]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
        
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=sent.MAX_LENGTH)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [87]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [88]:
def evaluate(encoder, decoder, sentence, max_length):
    with torch.no_grad():
        input_tensor = tensorFromSentence(sent, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(summ.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [89]:
def evaluateRandomly(encoder, decoder, n=10, max_length=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0], max_length)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [98]:
def evaluateRaw(encoder, decoder, sample, short, max_length):
    print('>', sample)
    print('=', short)
    output_words, attentions = evaluate(encoder, decoder, sample, max_length)
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print('')

In [90]:
hidden_size = 256
encoder1 = EncoderRNN(sent.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, sent.n_words, dropout_p=0.1, max_length=sent.MAX_LENGTH).to(device)

#trainIters(encoder1, attn_decoder1, 75000)
trainIters(encoder1, attn_decoder1, 2000, print_every=100, plot_every=100)

0m 11s (- 3m 29s) (100 5%) 3.6821
0m 21s (- 3m 12s) (200 10%) 2.9636
0m 32s (- 3m 6s) (300 15%) 2.5735
0m 44s (- 2m 58s) (400 20%) 2.3466
0m 56s (- 2m 49s) (500 25%) 1.9886
1m 11s (- 2m 47s) (600 30%) 1.6763
1m 38s (- 3m 3s) (700 35%) 1.1280
1m 52s (- 2m 48s) (800 40%) 0.6503
2m 4s (- 2m 32s) (900 45%) 0.4247
2m 17s (- 2m 17s) (1000 50%) 0.2229
2m 30s (- 2m 3s) (1100 55%) 0.1560
2m 42s (- 1m 48s) (1200 60%) 0.1204
2m 55s (- 1m 34s) (1300 65%) 0.0620
3m 7s (- 1m 20s) (1400 70%) 0.0507
3m 19s (- 1m 6s) (1500 75%) 0.0401
3m 29s (- 0m 52s) (1600 80%) 0.0317
3m 39s (- 0m 38s) (1700 85%) 0.0255
3m 48s (- 0m 25s) (1800 90%) 0.0232
3m 57s (- 0m 12s) (1900 95%) 0.0194
4m 7s (- 0m 0s) (2000 100%) 0.0177


In [103]:
sample="philippine share prices closed little changed monday as investors awaited the impact on oil prices from hurricane gustav as it slammed into the us gulf coast , dealers said ."
short="philippine shares close little changed"
evaluateRaw(encoder1, attn_decoder1, sample, short, max_length=40)

> bangladesh share prices closed little changed monday as investors awaited the impact on oil prices from hurricane gustav as it slammed into the us gulf coast , dealers said .
= bangladesh shares close little changed


RuntimeError: index out of range at /pytorch/aten/src/TH/generic/THTensorMath.cpp:352